### Deep Learning Final Project
#### Playing Modified Flappy Bird Using Reinforcement Learning
Leo Li(zl3493), Zhangnan Jiang(zj2028), Zichen Yang(zy2486) <br>
In this project, we would like to build our own reinforcement learning neural network to play the
Flappy Bird game. <br>
Please find all supporting files in the same folder. 

#### Importing necessary libraries:

In [1]:
import os
import cv2
import shutil
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchsummary
from random import random, randint, sample
from tensorboardX import SummaryWriter

#### Defining function used to preprocess game frame data (converting to grayscale):

In [2]:
def pre_processing(image, width, height):
    
    image = cv2.cvtColor(cv2.resize(image, (width, height)), cv2.COLOR_BGR2GRAY)
    _, image = cv2.threshold(image, 1, 255, cv2.THRESH_BINARY)
    return image[None, :, :].astype(np.float32)


#### Defining Model Structure:

Deep Q Network:

In [3]:
class DeepQNetwork(nn.Module):
    def __init__(self):
        super(DeepQNetwork, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(4, 32, kernel_size=8, stride=4), nn.ReLU(inplace=True))
        self.conv2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=4, stride=2), nn.ReLU(inplace=True))
        self.conv3 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.ReLU(inplace=True))

        self.fc1 = nn.Sequential(nn.Linear(7 * 7 * 64, 512), nn.ReLU(inplace=True))
        self.fc2 = nn.Linear(512, 2)
        
        self.init_weights()
        
    # Initiate model weights
    def init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                nn.init.uniform_(module.weight, -0.01, 0.01)
                nn.init.constant_(module.bias, 0)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)

        return x


Dueling DQN:

In [4]:
class DDQN(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(64)

        linear_input_size = 7 * 7 * 64
        fc_output_size=512
        self.fc_val=nn.Linear(linear_input_size, fc_output_size)
        self.fc_adv=nn.Linear(linear_input_size, fc_output_size)
        self.val = nn.Linear(fc_output_size, 1)
        self.adv = nn.Linear(fc_output_size, 2)

    def forward(self, x):
        x = x.float()
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        
        x_val = F.relu(self.fc_val(x.view(x.size(0), -1)))
        x_adv = F.relu(self.fc_adv(x.view(x.size(0), -1)))
        val=self.val(x_val)
        adv=self.adv(x_adv)
        
        x=val+adv-adv.mean(1,keepdim=True)
        return x

In [5]:
torch.cuda.is_available()

True

#### The training procedure:

Defining hyperparameters:

In [6]:
saved_path = "trained_models"
log_path = "scratch"
image_size = 84
lr = 1e-6
num_iters = 2000000
initial_epsilon = 0.1
final_epsilon = 1e-4
memory_buffer_size = 50000
gamma = 0.99
batch_size = 32

Load model select (can choose: Deep Q Network/ Dueling DQN, pretrain or not):

In [7]:
torch.cuda.manual_seed(123)
model = DeepQNetwork()
# model = torch.load("trained_models/flappy_bird")
# torchsummary.summary(model, (4, 8, 8))

Handling tensorboard log path:

In [8]:
if os.path.isdir(log_path):
        shutil.rmtree(log_path)
os.makedirs(log_path)
writer = SummaryWriter(log_path)

We can monitor the training progress by checking the embedded tensorboard: 

In [9]:
%load_ext tensorboard
%tensorboard --logdir scratch

Using Adam Optimizer and MSE loss between y_batch and q_value:

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

Starting the game and take the first frame as the initial input:

In [11]:
from src.flappy_bird import FlappyBird
game_state = FlappyBird() # start the game
image, reward, terminal = game_state.next_frame(0) # get the frame from the started game
image = pre_processing(image[:game_state.screen_width, :int(game_state.base_y)], image_size, image_size)
image = torch.from_numpy(image)

pygame 2.1.2 (SDL 2.0.16, Python 3.6.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [12]:
model.cuda()
image = image.cuda()
state = torch.cat(tuple(image for _ in range(4)))[None, :, :, :]

Initialize the relay buffer:

In [13]:
memory_buffer = []
iter = 0

The training iterations:

In [14]:
try:
    while iter < num_iters:
        
        prediction = model(state)[0]

        epsilon = final_epsilon + ((num_iters - iter) * (initial_epsilon - final_epsilon) / num_iters)
        iter += 1
        
        if random() <= epsilon:
            action = randint(0, 1)# make a random game action
        else:
            action = torch.argmax(prediction)
        
        # get the next frame from the started game applying a new game action
        next_image, reward, terminal = game_state.next_frame(action) 
        next_image = pre_processing(next_image[:game_state.screen_width, :int(game_state.base_y)], image_size, image_size) # preprocess the game frame
        next_image = torch.from_numpy(next_image)
        next_image = next_image.cuda()
        next_state = torch.cat((state[0, 1:, :, :], next_image))[None, :, :, :]
        
        memory_buffer.append([state, action, reward, next_state, terminal])
        
        if len(memory_buffer) > memory_buffer_size:
            del memory_buffer[0]
        
        batch = sample(memory_buffer, min(len(memory_buffer), batch_size))
        
        state_batch, action_batch, reward_batch, next_state_batch, terminal_batch = zip(*batch)

        state_batch = torch.cat(tuple(state for state in state_batch))
        action_batch = torch.from_numpy(np.array([[1, 0] if action == 0 else [0, 1] for action in action_batch], dtype=np.float32))
        reward_batch = torch.from_numpy(np.array(reward_batch, dtype=np.float32)[:, None])
        next_state_batch = torch.cat(tuple(state for state in next_state_batch))

        state_batch = state_batch.cuda()
        action_batch = action_batch.cuda()
        reward_batch = reward_batch.cuda()
        next_state_batch = next_state_batch.cuda()
        
        current_prediction_batch = model(state_batch)
        next_prediction_batch = model(next_state_batch)

        y_batch = torch.cat(tuple(reward if terminal else reward + gamma * torch.max(prediction) for reward, terminal, prediction in
zip(reward_batch, terminal_batch, next_prediction_batch)))

        q_value = torch.sum(current_prediction_batch * action_batch, dim=1)
        optimizer.zero_grad()

        loss = criterion(q_value, y_batch) # MSE
        loss.backward()
        optimizer.step()

        state = next_state
        
        if iter % 1000 == 0:
            print("Iteration: {}/{}, Action: {}, Loss: {}, Epsilon {}, Reward: {}, Q-value: {}".format(
                iter + 1,
                num_iters,
                action,
                loss,
                epsilon, reward, torch.max(prediction)))
            writer.add_scalar('Train/Loss', loss, iter)
            writer.add_scalar('Train/Epsilon', epsilon, iter)
            writer.add_scalar('Train/Reward', reward, iter)
            writer.add_scalar('Train/Q-value', torch.max(prediction), iter)

        if (iter+1) % 1000000 == 0:
            torch.save(model, "{}/fireball_flappy_bird_rewarded_scratch{}".format(saved_path, iter+1))
except KeyboardInterrupt:
    print("Saving model before quit")
    torch.save(model, "{}/fireball_flappy_bird_rewarded_scratch{}".format(saved_path, iter+1))

torch.save(model, "{}/fireball_flappy_bird_rewarded_scratch".format(saved_path))

Current FPS:  2.3348121643066406
Current FPS:  142.85714721679688
Current FPS:  147.05882263183594
Current FPS:  142.85714721679688
Current FPS:  144.9275360107422
Current FPS:  140.84506225585938
Current FPS:  147.05882263183594
Current FPS:  144.9275360107422
Current FPS:  149.25372314453125
Current FPS:  136.98629760742188
Current FPS:  147.05882263183594
Current FPS:  129.87013244628906
Current FPS:  138.88890075683594
Current FPS:  135.1351318359375
Current FPS:  128.2051239013672
Current FPS:  133.3333282470703
Current FPS:  136.98629760742188
Current FPS:  144.9275360107422
Current FPS:  147.05882263183594
Current FPS:  142.85714721679688
Current FPS:  133.3333282470703
Current FPS:  142.85714721679688
Iteration: 1001/2000000, Action: 1, Loss: 0.02870899811387062, Epsilon 0.09995009995000001, Reward: 0.1, Q-value: -0.09112685918807983
Current FPS:  133.3333282470703
Current FPS:  147.05882263183594
Current FPS:  142.85714721679688
Current FPS:  140.84506225585938
Current FPS:  1

Current FPS:  131.57894897460938
Current FPS:  136.98629760742188
Current FPS:  140.84506225585938
Current FPS:  135.1351318359375
Current FPS:  142.85714721679688
Current FPS:  138.88890075683594
Current FPS:  136.98629760742188
Current FPS:  140.84506225585938
Current FPS:  138.88890075683594
Current FPS:  135.1351318359375
Current FPS:  131.57894897460938
Current FPS:  129.87013244628906
Current FPS:  136.98629760742188
Current FPS:  140.84506225585938
Current FPS:  142.85714721679688
Current FPS:  147.05882263183594
Current FPS:  144.9275360107422
Current FPS:  144.9275360107422
Current FPS:  135.1351318359375
Current FPS:  136.98629760742188
Current FPS:  147.05882263183594
Current FPS:  142.85714721679688
Iteration: 11001/2000000, Action: 1, Loss: 0.00796377845108509, Epsilon 0.09945059995000001, Reward: 0.1, Q-value: -0.5193631052970886
Current FPS:  136.98629760742188
Current FPS:  140.84506225585938
Current FPS:  133.3333282470703
Current FPS:  136.98629760742188
Current FPS: 

Current FPS:  142.85714721679688
Current FPS:  142.85714721679688
Current FPS:  142.85714721679688
Current FPS:  136.98629760742188
Current FPS:  142.85714721679688
Current FPS:  140.84506225585938
Current FPS:  142.85714721679688
Current FPS:  149.25372314453125
Current FPS:  135.1351318359375
Current FPS:  133.3333282470703
Current FPS:  138.88890075683594
Current FPS:  136.98629760742188
Current FPS:  133.3333282470703
Current FPS:  136.98629760742188
Current FPS:  138.88890075683594
Current FPS:  149.25372314453125
Current FPS:  140.84506225585938
Current FPS:  138.88890075683594
Current FPS:  140.84506225585938
Current FPS:  140.84506225585938
Current FPS:  140.84506225585938
Current FPS:  147.05882263183594
Iteration: 21001/2000000, Action: 1, Loss: 0.01594548299908638, Epsilon 0.09895109995000001, Reward: 0.1, Q-value: 0.8686089515686035
Current FPS:  142.85714721679688
Current FPS:  140.84506225585938
Current FPS:  140.84506225585938
Current FPS:  128.2051239013672
Current FPS:

Current FPS:  138.88890075683594
Current FPS:  140.84506225585938
Iteration: 30001/2000000, Action: 0, Loss: 0.00410010851919651, Epsilon 0.09850154995, Reward: 0.1, Q-value: 0.7788987159729004
Current FPS:  138.88890075683594
Current FPS:  140.84506225585938
Current FPS:  140.84506225585938
Current FPS:  140.84506225585938
Current FPS:  142.85714721679688
Current FPS:  140.84506225585938
Current FPS:  140.84506225585938
Current FPS:  147.05882263183594
Current FPS:  142.85714721679688
Current FPS:  144.9275360107422
Current FPS:  142.85714721679688
Current FPS:  147.05882263183594
Current FPS:  147.05882263183594
Current FPS:  144.9275360107422
Current FPS:  144.9275360107422
Current FPS:  138.88890075683594
Current FPS:  136.98629760742188
Current FPS:  140.84506225585938
Current FPS:  135.1351318359375
Current FPS:  136.98629760742188
Current FPS:  135.1351318359375
Iteration: 31001/2000000, Action: 1, Loss: 0.005686512216925621, Epsilon 0.09845159995000001, Reward: 0.1, Q-value: -0

Current FPS:  133.3333282470703
Current FPS:  147.05882263183594
Current FPS:  147.05882263183594
Current FPS:  149.25372314453125
Current FPS:  142.85714721679688
Current FPS:  147.05882263183594
Current FPS:  147.05882263183594
Current FPS:  136.98629760742188
Current FPS:  138.88890075683594
Iteration: 40001/2000000, Action: 0, Loss: 0.0037964400835335255, Epsilon 0.09800204995, Reward: 0.1, Q-value: 1.3763221502304077
Current FPS:  138.88890075683594
Current FPS:  135.1351318359375
Current FPS:  136.98629760742188
Current FPS:  136.98629760742188
Current FPS:  133.3333282470703
Current FPS:  136.98629760742188
Current FPS:  144.9275360107422
Current FPS:  136.98629760742188
Current FPS:  131.57894897460938
Current FPS:  138.88890075683594
Current FPS:  135.1351318359375
Current FPS:  138.88890075683594
Current FPS:  149.25372314453125
Current FPS:  140.84506225585938
Current FPS:  125.0
Current FPS:  138.88890075683594
Current FPS:  138.88890075683594
Current FPS:  140.845062255859

...

Iteration: 1885001/2000000, Action: 0, Loss: 0.007576029747724533, Epsilon 0.00584429995, Reward: 0.1, Q-value: -0.18505212664604187
Current FPS:  138.88890075683594
Current FPS:  142.85714721679688
Current FPS:  142.85714721679688
Current FPS:  140.84506225585938
Current FPS:  142.85714721679688
Current FPS:  140.84506225585938
Current FPS:  142.85714721679688
Iteration: 1886001/2000000, Action: 1, Loss: 0.0073090773075819016, Epsilon 0.00579434995, Reward: 0.1, Q-value: 0.015282225795090199
Current FPS:  133.3333282470703
Current FPS:  136.98629760742188
Current FPS:  133.3333282470703
Current FPS:  142.85714721679688
Iteration: 1887001/2000000, Action: 0, Loss: 0.011446438729763031, Epsilon 0.00574439995, Reward: 0.1, Q-value: 0.017915867269039154
Current FPS:  133.3333282470703
Current FPS:  138.88890075683594
Current FPS:  136.98629760742188
Current FPS:  129.87013244628906
Current FPS:  129.87013244628906
Current FPS:  133.3333282470703
Iteration: 1888001/2000000, Action: 1, Loss

Current FPS:  135.1351318359375
Current FPS:  131.57894897460938
Current FPS:  135.1351318359375
Current FPS:  135.1351318359375
Current FPS:  140.84506225585938
Iteration: 1911001/2000000, Action: 0, Loss: 0.008809315040707588, Epsilon 0.00454559995, Reward: 0.1, Q-value: 2.698599338531494
Current FPS:  133.3333282470703
Current FPS:  133.3333282470703
Iteration: 1912001/2000000, Action: 0, Loss: 0.007170794066041708, Epsilon 0.0044956499500000005, Reward: 0.1, Q-value: -0.005289943888783455
Current FPS:  135.1351318359375
Current FPS:  135.1351318359375
Current FPS:  131.57894897460938
Current FPS:  128.2051239013672
Current FPS:  126.582275390625
Current FPS:  136.98629760742188
Current FPS:  140.84506225585938
Iteration: 1913001/2000000, Action: 1, Loss: 0.012133970856666565, Epsilon 0.0044456999500000005, Reward: 0.1, Q-value: 2.3020992279052734
Current FPS:  140.84506225585938
Current FPS:  135.1351318359375
Current FPS:  136.98629760742188
Current FPS:  142.85714721679688
Iterat

Current FPS:  131.57894897460938
Iteration: 1935001/2000000, Action: 0, Loss: 0.01017870381474495, Epsilon 0.00334679995, Reward: 0.1, Q-value: 0.12775973975658417
Current FPS:  129.87013244628906
Current FPS:  129.87013244628906
Current FPS:  125.0
Current FPS:  138.88890075683594
Iteration: 1936001/2000000, Action: 0, Loss: 0.008576353080570698, Epsilon 0.0032968499500000002, Reward: 0.1, Q-value: 2.038522481918335
Current FPS:  142.85714721679688
Current FPS:  138.88890075683594
Current FPS:  136.98629760742188
Current FPS:  135.1351318359375
Current FPS:  133.3333282470703
Current FPS:  138.88890075683594
Current FPS:  135.1351318359375
Current FPS:  138.88890075683594
Iteration: 1937001/2000000, Action: 0, Loss: 0.00879382248967886, Epsilon 0.00324689995, Reward: 0.1, Q-value: 4.7393574714660645
Current FPS:  129.87013244628906
Current FPS:  136.98629760742188
Current FPS:  131.57894897460938
Current FPS:  138.88890075683594
Current FPS:  140.84506225585938
Current FPS:  138.88890

Current FPS:  142.85714721679688
Current FPS:  133.3333282470703
Iteration: 1963001/2000000, Action: 0, Loss: 0.010996675118803978, Epsilon 0.00194819995, Reward: 0.1, Q-value: 1.8473515510559082
Current FPS:  138.88890075683594
Current FPS:  133.3333282470703
Current FPS:  133.3333282470703
Iteration: 1964001/2000000, Action: 0, Loss: 0.010788941755890846, Epsilon 0.0018982499500000002, Reward: 0.1, Q-value: 4.920207500457764
Current FPS:  131.57894897460938
Current FPS:  142.85714721679688
Current FPS:  129.87013244628906
Current FPS:  136.98629760742188
Iteration: 1965001/2000000, Action: 1, Loss: 0.00850982591509819, Epsilon 0.0018482999500000002, Reward: 0.1, Q-value: -1.1001907587051392
Current FPS:  133.3333282470703
Current FPS:  133.3333282470703
Current FPS:  136.98629760742188
Current FPS:  133.3333282470703
Current FPS:  144.9275360107422
Current FPS:  147.05882263183594
Current FPS:  135.1351318359375
Current FPS:  131.57894897460938
Iteration: 1966001/2000000, Action: 0, 

Iteration: 1990001/2000000, Action: 0, Loss: 0.009373166598379612, Epsilon 0.0005995499500000001, Reward: 0.1, Q-value: 2.792339563369751
Current FPS:  126.582275390625
Current FPS:  109.8901138305664
Current FPS:  142.85714721679688
Current FPS:  133.3333282470703
Current FPS:  121.95121765136719
Iteration: 1991001/2000000, Action: 0, Loss: 0.008376523852348328, Epsilon 0.00054959995, Reward: 0.1, Q-value: 5.396687984466553
Current FPS:  140.84506225585938
Current FPS:  128.2051239013672
Current FPS:  131.57894897460938
Current FPS:  133.3333282470703
Current FPS:  120.48193359375
Current FPS:  123.456787109375
Current FPS:  133.3333282470703
Iteration: 1992001/2000000, Action: 0, Loss: 0.012004777789115906, Epsilon 0.00049964995, Reward: 0.1, Q-value: 1.7956023216247559
Current FPS:  128.2051239013672
Current FPS:  112.35955047607422
Current FPS:  140.84506225585938
Current FPS:  140.84506225585938
Current FPS:  138.88890075683594
Current FPS:  128.2051239013672
Iteration: 1993001/20

#### Test Procedure:

In [ ]:
saved_path = "trained_models"
image_size = 84

In [ ]:
torch.cuda.manual_seed(123)

In [ ]:
model = torch.load("{}/fireball_flappy_bird".format(saved_path))

model.eval()
try:
    from src.flappy_bird import FlappyBird
    game_state = FlappyBird()
    image, reward, terminal = game_state.next_frame(0)
    image = pre_processing(image[:game_state.screen_width, :int(game_state.base_y)], image_size, image_size)
    image = torch.from_numpy(image)
    if torch.cuda.is_available():
        model.cuda()
        image = image.cuda()
    state = torch.cat(tuple(image for _ in range(4)))[None, :, :, :]

    while True:
        prediction = model(state)[0]
        action = torch.argmax(prediction)

        next_image, reward, terminal = game_state.next_frame(action)
        next_image = pre_processing(next_image[:game_state.screen_width, :int(game_state.base_y)], image_size,
                                    image_size)
        next_image = torch.from_numpy(next_image)
        if torch.cuda.is_available():
            next_image = next_image.cuda()
        next_state = torch.cat((state[0, 1:, :, :], next_image))[None, :, :, :]

        state = next_state
except KeyboardInterrupt:
    print("Quit")